In this example we will create alkyl chains of various lengths, run quantum chemical analysis on each and then replicate them into a simulation cell for an MD simulation

In [1]:
import os 
from pprint import pprint

Check that output from other examples has been generated 

In [2]:
from pathlib2 import Path

In [3]:
need_files = ['methane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run structures_example.ipynb")
        os.system("jupyter nbconvert --to python  structures_example.ipynb")
        os.system("python structures_example.py")

In [4]:
need_files = ['ethane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run buildingblocks_example.ipynb")
        os.system("jupyter nbconvert --to python  buildingblocks_example.ipynb")
        os.system("python buildingblocks_example.py")

In [5]:
need_files = ['oplsaa_param.json']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields_example.ipynb")
        os.system("jupyter nbconvert --to python  forcefields_example.ipynb")
        os.system("python forcefields_example.py")

In [6]:
import streamm

Now let's create project and resource to keep track of our work

In [7]:
alkyl_example = streamm.Project('alkyl_example')
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [8]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [9]:
print res_local.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Create the local directories that will store our files 

In [10]:
res_local.make_dir()

Tell the project about our directories 

In [11]:
alkyl_example.set_resource(res_local)

Read in the methane.xyz file created in the structure_example.ipynb example 

In [12]:
methane = streamm.Buildingblock('methane')

In [13]:
methane.read_xyz()

Create the neighbor list 

In [14]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

and the bonded interactions

In [15]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [16]:
print methane.n_particles

5


In [17]:
print methane.print_properties()

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


Set the `paramkeys` so we can identify force field parameters later on

In [18]:
for pkey,p in methane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [19]:
for pk,p in methane.particles.iteritems():
    p.residue = 1
    p.resname = 'METH'

Set some `rsites` to be able to join molecules together 

In [20]:
methane.particles[1].rsite = 'RH'
methane.particles[2].rsite = 'RH'

In [21]:
methane.find_rsites()

In [22]:
print methane.show_rsites()

rsite:RH[ paticle:atom H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom H (H) index:2 n_bonds:1] 



Read in ethane.xyz from the buildinblock_example.ipynb example 

In [23]:
ethane = streamm.Buildingblock('ethane')

In [24]:
ethane.read_xyz()

Guess bonded neighbor list based on `bonded_radii`

In [25]:
ethane.bonded_nblist = ethane.guess_nblist(0,radii_buffer=1.25)

In [26]:
ethane.bonded_bonds()
ethane.bonded_angles()
ethane.bonded_dih()

In [27]:
print ethane.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


Set the `paramkey`'s  as described in the force field example

In [28]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

Set the `resname` of each particle to `ETH` 

In [29]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

Set `rsite`'s to hydrogens to be replaced during join

In [30]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

Run `find_rsites()` to populate `func` list

In [31]:
ethane.find_rsites()

In [32]:
print ethane.show_rsites()

rsite:RH[ paticle:atom H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom H (H) index:5 n_bonds:1] 



In [33]:
import copy

Create octane from ethane 

Copy ethane to a new Buildingblock octane 

In [34]:
octane = copy.deepcopy(ethane)

In [35]:
from streamm.structures.buildingblock import attach

Then attach 3 more ethanes to make an octane 

In [36]:
for i in range(3):
    octane = attach(octane,ethane,'RH',1,'RH',0)

Update the tag

In [37]:
octane.tag = 'octane'

Rename the residue and resname for octane

In [38]:
for pk,p in octane.particles.iteritems():
    p.residue = 2
    p.resname = "OCT"
 

In [39]:
octane.write_xyz()

Print new `rsite`'s 

In [40]:
print octane.show_rsites()

rsite:RH[ paticle:atom H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom H (H) index:23 n_bonds:1] 



Find the 4th carbon to attach an ethane 

In [41]:
print octane.particles[14].symbol

H


In [42]:
octane.particles[14].rsite = 'R2'

In [43]:
octane.find_rsites()

Attach the ethane to the fourth carbon to make 4-ethyloctane

In [44]:
ethyl_octane = attach(octane,ethane,'R2',0,'RH',0)

In [45]:
ethyl_octane.tag = '4-ethyloctane'

In [46]:
ethyl_octane.write_xyz()

Read in oplsaa parameters from forcefield example 

In [47]:
oplsaa = streamm.forcefields.parameters.Parameters('oplsaa')

In [48]:
oplsaa.import_json()

In [49]:
print oplsaa


    Parameters 
      LJ parameters 2 
      Bond parameters 2 
      Angle parameters 2 
      Dihedral parameters 1 
      Improper Dihedral parameters 0 



In [50]:
for pk,ptypes in oplsaa.particletypes.iteritems():
    print ptypes.fftype1

CT
HC


Create NWChem Calculation object 

In [51]:
nwchem_i = streamm.NWChem('nw_ethane_HF')

Add calculation to project

In [52]:
alkyl_example.add_calc(nwchem_i)

Set the structure of the calculation to ethane

In [53]:
nwchem_i.strucC = ethane

Set the resource to be local

In [54]:
nwchem_i.set_resource(res_local)

Make the local directories 

In [55]:
nwchem_i.make_dir()

Change to the `scratch` directory

In [56]:
os.chdir(nwchem_i.dir['scratch'])

Copy the template files to the scratch directory 

In [57]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [58]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [59]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [60]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_i.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [61]:
pprint(nwchem_i.properties)

{u'allocation': u'',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/nw_ethane_HF/',
 u'spin_mult': 1,
 u'task': 'SCF ',
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


Replace the keys in the template strings and write the input files 

In [62]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

Add the input file to the properties to be written into the run file

In [63]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_i.tag))

Add the log file to the files dictionary 

In [64]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [65]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.export_json()

{u'calculations': {'nw_ethane_HF': u'nwchem'},
 u'meta': {'date': '2017-11-15T17:22:59.842206',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['local']}

Change back to scratch

In [66]:
os.chdir(nwchem_i.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [67]:
nwchem_i.run()

Check the status of all the calculations in the project

In [68]:
alkyl_example.check()

Calculation nw_ethane_HF has status running


Run the analysis

In [69]:
nwchem_i.analysis()

File nw_ethane_HF.log not found 


Tar and zip the results and copy them to a storage location

In [70]:
nwchem_i.store()

Save json in home directory

In [71]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.export_json()

{u'calculations': {'nw_ethane_HF': u'nwchem'},
 u'meta': {'date': '2017-11-15T17:22:59.842206',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['local']}

Create a Gaussian Calculation object 

In [72]:
gaussian_i = streamm.Gaussian('gaus_ethane_HF')

Add the calculation to the project

In [73]:
alkyl_example.add_calc(gaussian_i)

Set the structure of the calculation to ethane

In [74]:
gaussian_i.strucC = ethane

Set the resource to be local

In [75]:
gaussian_i.set_resource(res_local)

Make the local directories 

In [76]:
gaussian_i.make_dir()

Copy the template files to the scratch directory 

In [77]:
os.chdir(gaussian_i.dir['scratch'])

Copy the template files to the scratch directory 

In [78]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [79]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [80]:
gaussian_i.load_str('templates','com')        
gaussian_i.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [81]:
gaussian_i.properties['commands'] = 'HF/3-21G SP'
gaussian_i.properties['method'] = 'UHF'
gaussian_i.properties['charge'] = 0
gaussian_i.properties['spin_mult'] = 1
gaussian_i.properties['coord'] = gaussian_i.strucC.write_coord()

In [82]:
pprint(gaussian_i.properties)

{u'allocation': u'',
 'charge': 0,
 'commands': 'HF/3-21G SP',
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Normal termination of Gaussian',
 'method': 'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/gaus_ethane_HF/',
 'spin_mult': 1,
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


Replace the keys in the template strings and write the input files 

In [83]:
gaussian_i.replacewrite_prop('com','input','com','%s.com'%(gaussian_i.tag))

Add the input file to the properties to be written into the run file

In [84]:
gaussian_i.properties['input_com'] = gaussian_i.files['input']['com']
gaussian_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaussian_i.tag))

Add the log file to the files dictionary 

In [85]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaussian_i.tag)
gaussian_i.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [86]:
os.chdir(gaussian_i.dir['home'])
alkyl_example.export_json()

{u'calculations': {'gaus_ethane_HF': u'gaussian', 'nw_ethane_HF': u'nwchem'},
 u'meta': {'date': '2017-11-15T17:22:59.842206',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['local']}

Change back to scratch

In [87]:
os.chdir(gaussian_i.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [88]:
gaussian_i.run()

Check the status of all the calculations in the project

In [89]:
alkyl_example.check()

Calculation nw_ethane_HF has status running
Calculation gaus_ethane_HF has status running


Run the analysis

In [90]:
os.chdir(alkyl_example.dir['home'])
alkyl_example.export_json()

{u'calculations': {'gaus_ethane_HF': u'gaussian', 'nw_ethane_HF': u'nwchem'},
 u'meta': {'date': '2017-11-15T17:22:59.842206',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['local']}

Create a LAMMPS Calculation object 

In [91]:
lmp_alkyl = streamm.LAMMPS('lmp_alkyl')

Turn periodic boundaries on in all three directions

In [92]:
lmp_alkyl.strucC.lat.pbcs = [True,True,True]

Run the `add_struc()` function to create 10 randomly placed 4-ethyloctane molecules

In [93]:
seed = 92734
lmp_alkyl.strucC = streamm.add_struc(lmp_alkyl.strucC,ethyl_octane,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


The `add_struc()` function randomly places each molecule in a space defined by the lattice of the lmp_alkyl.strucC, then randomly rotates it. 

Then the function checks to make sure it does not overlap any other particles that are already in the lmp_alkyl.strucC. 

If an overlap is found a new position and rotation is chosen until the max placements are exceeded, then the entire system is cleared, and the placement starts again. If the maximum restarts are exceeded, then the size of the lattice is increased, until all the molecules have been added.

Check the lattice to see if it expanded 

In [94]:
print lmp_alkyl.strucC.lat

100.000000 0.000000 0.000000
0.000000 100.000000 0.000000
0.000000 0.000000 100.000000


Find the maximum molecule index 

In [95]:
print lmp_alkyl.strucC.n_molecules()

9


In [96]:
print ethyl_octane.tag

4-ethyloctane


Update the structure tag

In [97]:
lmp_alkyl.strucC.tag = ethyl_octane.tag + '_x10'

Write the structure to an xyz file

In [98]:
lmp_alkyl.strucC.write_xyz()

Add 10 ethanes to the structure container

In [99]:
seed = 283674
lmp_alkyl.strucC = streamm.add_struc(lmp_alkyl.strucC,ethane,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


In [100]:
print lmp_alkyl.strucC.n_molecules()

19


Update tag

In [101]:
lmp_alkyl.strucC.tag += '_ethane_x10'

Add 50 methane to structure container using the `add_struc_grid()`, which places solvent on grid 

In [102]:
lmp_alkyl.strucC = streamm.add_struc_grid(lmp_alkyl.strucC,methane,50)

Check to see if the lattice was expanded 

In [103]:
print lmp_alkyl.strucC.lat

100.000000 0.000000 0.000000
0.000000 100.000000 0.000000
0.000000 0.000000 100.000000


Update tag

In [104]:
lmp_alkyl.strucC.tag += '_methane_x50'

In [105]:
lmp_alkyl.strucC.write_xyz()

Print all the particles in the structure container 

In [106]:
for pk,p in lmp_alkyl.strucC.particles.iteritems():
    print p,p.paramkey,p.mol,p.residue,p.resname

atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom H (H) HC 0 2 OCT
atom C (C) CT 0 1 ETH
atom H (H) HC 0 1 ETH
atom H (H) HC 0 1 ETH
atom C (C) CT 0 1 ETH
atom H (H) HC 0 1 ETH
atom H (H) HC 0 1 ETH
atom H (H) HC 0 1 ETH
atom C (C) CT 1 2 OCT
atom H (H) HC 1 2 OCT
atom H (H) HC 1 2 OCT
atom H (H) HC 1 2 OCT
atom C (C) CT 1 2 OCT
atom H (H) HC 1 2 OCT
atom H (H) HC 1 2 OCT
atom C (C) CT 1 2 OCT
atom H (H) HC 1 2 OCT
atom H (H) HC 1 2 OCT
atom C (C) CT 1 2 OCT
atom H (H) HC 1 2 OCT
atom H (H) HC 1 2 OCT
atom C (C)

Set ff parameters for all the bonds, bond angles and dihedrals in the structure container

In [107]:
lmp_alkyl.paramC = oplsaa

In [108]:
lmp_alkyl.set_ffparam()

Add template files to calculations

In [109]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_spneut.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_alkyl.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [110]:
pprint("Calculation:{} has status:{}".format(lmp_alkyl.tag,lmp_alkyl.meta['status']))

'Calculation:lmp_alkyl has status:written'


Calculate the center mass of structure

In [111]:
lmp_alkyl.strucC.calc_center_mass()

Create groups out of the molecules

In [112]:
groupset_i = streamm.Groups('mol',lmp_alkyl.strucC)
groupset_i.group_prop('mol','group_mol')

Caculate the ceneter of mass, radius and asphericity of each group

In [113]:
groupset_i.calc_cent_mass()
groupset_i.calc_radius_asphericity()
groupset_i.calc_dl()

Write the center of mass of each group to an .xyz file for visualization 

In [114]:
groupset_i.write_cm_xyz()

In [115]:
import numpy as np

In [116]:
print np.mean(groupset_i.radius),groupset_i.strucC.unit_conf['length']

1.79932546227 ang


In [117]:
print groupset_i.strucC.lat.pbcs

[True, True, True]


Create a neighbor list of groups 

In [118]:
groupset_i.group_nblist.radii_nblist(groupset_i.strucC.lat,groupset_i.cent_mass,groupset_i.radius,radii_buffer=5.25)

Apply periodic boundaries to all the groups, so the molecules are not split across pbc's

In [119]:
groupset_i.group_pbcs()

Loop over each group, shift the group to the center of the simulation cell and write an .xyz file that includes the neighbors of the group.

In [120]:
for gk_i,g_i in groupset_i.groups.iteritems():
    if( len(g_i.pkeys) == 32 ):
        print g_i.tag,groupset_i.group_nblist.calc_nnab(gk_i),g_i.mol 
        print g_i.cent_mass
        list_i = []
        for g_j in groupset_i.group_nblist.getnbs(gk_i):
            list_i += groupset_i.groups[g_j].pkeys
        groupset_i.strucC.shift_pos(-1.0*g_i.cent_mass)  # Place center of mass at origin
        groupset_i.strucC.write_xyz_list(list_i,xyz_file='{}_blob.xyz'.format(g_i.tag))
        groupset_i.strucC.shift_pos(g_i.cent_mass)  # Return center of mass 
        

group_mol_0 16 0
[ 76.653846  10.30594    0.623454]
group_mol_1 13 1
[ 73.198925  53.63841   74.637648]
group_mol_2 17 2
[  6.854253  86.197885  25.639487]
group_mol_3 14 3
[  4.771635  68.191421  89.863634]
group_mol_4 17 4
[ 25.9855    36.503382  59.348536]
group_mol_5 15 5
[ 98.939799  17.188399  88.616208]
group_mol_6 13 6
[ 34.513904   5.223367  49.861893]
group_mol_7 9 7
[ 66.215706  71.928804  48.970751]
group_mol_8 14 8
[ 98.116022  81.682274   1.600691]
group_mol_9 13 9
[ 36.312087  57.242611  93.212946]


Fancy aye! 